# An introduction to JuMP

This tutorial doesn't exist in isolation.
Some other good resources for learning JuMP are:
- [Julia Discourse forum](https://discourse.julialang.org/c/domain/opt)
- [JuMP documentation](http://www.juliaopt.org/JuMP.jl/0.18)
- [JuMP examples](https://github.com/JuliaOpt/JuMP.jl/tree/release-0.18/examples)
- [Textbook: Julia Programming for Operations Research](http://www.chkwon.net/julia)

## The basics

First, load the JuMP package into your current environment.

In [2]:
using JuMP

Start building a JuMP model like so,

In [3]:
model = Model()
@variable(model, x)
@variable(model, y >= 0)
@variable(model, 1 <= z <= 2)
model

A JuMP Model
Feasibility problem with:
Variables: 3
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x, y, z

### What's going on here?
`@variable(model, x)` does four things:
1. it adds an *optimization* variable to the model
2. it creates a *JuMP* variable that acts as a reference to the optimization variable in the model
3. it creates a *Julia* variable `x` that points to the JuMP variable
4. it stores a reference to the JuMP variable in the model with the name `:x`

In [4]:
model = Model()
@variable(model, x >= 1.414)

x

In [5]:
# x is a JuMP variable
typeof(x)

VariableRef

In [6]:
# We can bind the JuMP variable to a different Julia variable and set `x` to something else
y = x
x = 1

@show typeof(y)
@show typeof(x)

y

typeof(y) = VariableRef
typeof(x) = Int64


x

In [7]:
JuMP.lower_bound(y)

1.414

In [8]:
model[:x]

x

In [9]:
model[:x] == y

true

### Other ways to create variables

We can also create arrays of JuMP variables.

In [10]:
model = Model()
@variable(model, x[i = 1:4] >= i)
x

4-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]

The indices of the arrays don't have to be integers. They can be anything, like a string or a symbol.

In [11]:
model = Model()
@variable(model, x[i = 1:2, j = [:A, :B]] >= i)
x

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Base.OneTo(2)
    Dimension 2, [:A, :B]
And data, a 2×2 Matrix{VariableRef}:
 x[1,A]  x[1,B]
 x[2,A]  x[2,B]

We can also add a condition to filter out some of the variables

In [12]:
model = Model()
@variable(model, x[i = 1:4, j = [:A, :B]; isodd(i)] >= i)
x

JuMP.Containers.SparseAxisArray{VariableRef, 2, Tuple{Int64, Symbol}} with 4 entries:
  [1, A]  =  x[1,A]
  [1, B]  =  x[1,B]
  [3, A]  =  x[3,A]
  [3, B]  =  x[3,B]

What if I want to add two variables with the same name?

In [13]:
model = Model()
@variable(model, x >= 1)
@variable(model, x >= 2)

LoadError: An object of name x is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, e.g.,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :x)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:x]`.


### Quiz Question

What is the value of the following?

In [14]:
JuMP.lower_bound(x)

1.0

### Anonymous variables

So far, we've only added *named* variables. This works great until we want to add two variables with the same name! To work around this isse, we can add *anonymous* variables.

In [15]:
model = Model()

x = @variable(model)
@show JuMP.has_lower_bound(x)

x = @variable(model, lower_bound = 1)
@show JuMP.lower_bound(x)

x = @variable(model, [i = 1:2], lower_bound = i)
@show JuMP.lower_bound.(x)

model

JuMP.has_lower_bound(x) = false
JuMP.lower_bound(x) = 1.0
JuMP.lower_bound.(x) = [1.0, 2.0]


A JuMP Model
Feasibility problem with:
Variables: 4
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

So what's the difference between the *named* and *anonymous* syntax? 

Well, `@variable(model, x >= 2)` is (roughly) equivalent to

In [16]:
model = Model()
x = model[:x] = @variable(model, lower_bound = 2, base_name = "x")
model

A JuMP Model
Feasibility problem with:
Variable: 1
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

In [17]:
model = Model()
@variable(model, x >= 2)
model

A JuMP Model
Feasibility problem with:
Variable: 1
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

Although uneeded for this tutorial, we can also create binary and integer variables as follows:

In [18]:
model = Model()
@variable(model, x >= 1, Int)
@variable(model, y, Bin)
model

A JuMP Model
Feasibility problem with:
Variables: 2
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.Integer`: 1 constraint
`VariableRef`-in-`MathOptInterface.ZeroOne`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x, y

### Constraints

Now that we've seen how to create variables, let's look at constraints. Much of the syntax should be familiar.

In [19]:
model = Model()
@variable(model, x >= 0)
@variable(model, y >= 0)

@constraint(model, c_less_than, 2x + y <= 1)
@constraint(model, c_greater_than, 2x + y >= 1)
@constraint(model, c_equal_to, 2x + y == 1)

model

A JuMP Model
Feasibility problem with:
Variables: 2
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: c_equal_to, c_greater_than, c_less_than, x, y

In [20]:
model[:c_equal_to]

c_equal_to : 2 x + y = 1.0

In [21]:
anonymous_constraint = @constraint(model, [i = 1:2], i * x <= y)

model

A JuMP Model
Feasibility problem with:
Variables: 2
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: c_equal_to, c_greater_than, c_less_than, x, y

### Objective Functions

Now let's look at the objective function.

In [22]:
model = Model()
@variable(model, x >= 0)

@objective(model, Min, 2x + 1)

model

A JuMP Model
Minimization problem with:
Variable: 1
Objective function type: AffExpr
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

In [23]:
model = Model()
@variable(model, x <= 2)

@objective(model, Max, 2x + 1)

model

A JuMP Model
Maximization problem with:
Variable: 1
Objective function type: AffExpr
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

### Solving a Model

Once we've formulated a model, the next step is to solve it. This requires a solver.

JuMP supports lots of different solvers. The [JuMP documentation](http://www.juliaopt.org/JuMP.jl/v0.19.2/installation/) contains a list of the supported solvers and the types of problems each solver supports.

We're going to use two solvers in particular.

The first solver is the [GNU Linear Programming Kit (GLPK)](https://www.gnu.org/software/glpk/). This solver supports linear programs with continous variables.

GLPK is available via the [GLPK.jl](https://github.com/JuliaOpt/GLPK.jl) package.

The second solver is the COIN-OR [Interior Point OPTimizer (Ipopt)](https://projects.coin-or.org/Ipopt). This solver supports nonlinear programs with continous variables.

Ipopt is available via the [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) package.

In [25]:
using GLPK
using Ipopt

There are two ways to add a solver to a JuMP model:

In [26]:
model = Model(GLPK.Optimizer)

# ... or ...

model = Model()
set_optimizer(model, Ipopt.Optimizer)

If you try to solve an unsupported problem type, an error will be thrown:

In [27]:
model = Model(GLPK.Optimizer)
@variable(model, 0 <= x <= π)
@NLobjective(model, Min, cos(x)^2)
optimize!(model)

LoadError: The solver does not support nonlinear problems (i.e., NLobjective and NLconstraint).

That error isn't very nice, but there is an [open JuMP issue](https://github.com/JuliaOpt/JuMP.jl/issues/1996) to resolve this.

In [28]:
set_optimizer(model, Ipopt.Optimizer)
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equal

## Interpreting statuses

After solving a model, JuMP can report three different statuses:

- `termination_status(model)` explains why the solver stopped. Common statuses are `OPTIMAL`, `INFEASIBLE`, `DUAL_INFEASIBLE` (i.e., primal is potentially unbounded), and `LOCALLY_SOLVED`.

- `primal_status(model)` explains how to interpret the primal solution vector. Common statuses are `FEASIBLE_POINT` and `NO_SOLUTION`.

- `dual_status(model)` explains how to interpret the dual solution vector. Common statuses are `FEASIBLE_POINT` and `NO_SOLUTION`.

## Getting solutions

- Use `objective_value(::Model)` to get the objective value
- Use `value(::VariableRef)` to get the primal value of a variable
- Use `dual(::ConstraintRef)` to get the value of the dual variable associated with a constraint

In [29]:
x_value = value(x)
obj_value = objective_value(model)

println("Termination status: ", termination_status(model))
println("Primal status:      ", primal_status(model))
println("Dual status:        ", dual_status(model))
println("      x | $(x_value)")
println("    π/2 | $(π/2)")
println("--------+----------------------")
println("cos²(x) | $(obj_value)")

Termination status: LOCALLY_SOLVED
Primal status:      FEASIBLE_POINT
Dual status:        FEASIBLE_POINT
      x | 1.5707963267948966
    π/2 | 1.5707963267948966
--------+----------------------
cos²(x) | 3.749399456654644e-33


## Example: Simple Economic Dispatch

*This example is adapted from a [tutorial given at the 2015 Grid Science Winter School](https://github.com/JuliaOpt/juliaopt-notebooks/blob/3110eddaf5effdecfee687739295bea05731ba33/notebooks/Dvorkin%20-%20Power%20systems%20-%20Economic%20dispatch%20and%20Unit%20commitment.ipynb).*

Economic dispatch (ED) is an optimization problem that minimizes the cost of supplying energy demand subject to operational constraints on power system assets. In its simplest instantiation, ED is a Linear Programing (LP) problem solved for an aggregated load and wind forecast and for a single infinitesimal moment. Mathematically, the ED problem can be written as follows:
$$
\min \sum_{i \in I} c^g_{i} \cdot g_{i} + c^w \cdot w,
$$
where $c_{i}$ and $g_{i}$ are the incremental cost ($\$/MWh$) and power output ($MW$) of the $i^{th}$ thermal generator, respectively, and $c^w$ and $w$ are the incremental cost ($\$/MWh$) and wind power injection ($MW$), respectively.

s.t.

<li> Minimum ($g^{\min}$) and maximum ($g^{\max}$) limits on power outputs of the thermal generators: </li>
$$
g^{\min}_{i} \leq g_{i} \leq g^{\max}_{i}.
$$
<li>Constraint on the wind power injection:</li>
$$
0 \leq w \leq w^f, 
$$
where $w$ and $w^f$ are the wind power injection and wind power forecast, respectively.

<li>Power balance constraint:</li>
$$
\sum_{i \in I} g_{i} + w = d, 
$$
where $d$ is the demand.

Further reading on ED models can be found in A. J. Wood, B. F. Wollenberg, and G. B. Sheblé, "Power Generation, Operation and Control", Wiley, 2013.

### JuMP Implementation of Economic Dispatch 

First, we need to load some plotting packages.  Note, compilation may take a few minutes.

In [30]:
using Interact
using Plots

LoadError: ArgumentError: Package Interact not found in current path:
- Run `import Pkg; Pkg.add("Interact")` to install the Interact package.


Now, we define some problem data for a model with two thermal generators and one wind generator.

In [ ]:
# Define some input data about the test system
# Maximum power output of generators
const GENERATION_MAX = [1000, 1000]
# Minimum power output of generators
const GENERATION_MIN = [0, 300]
# Incremental cost of generators 
const COST_GENERATION = [50, 100]
# Incremental cost of wind generators
const COST_WIND = 50
# Total demand
const DEMAND = 1500
# Wind forecast
const WIND_MAX = 200;

In the next cell, we create a Julia function that formulates and solves the economic dispatch problem.

In [ ]:
"""
    solve_economic_dispatch(; cost_of_thermal::Vector, cost_of_wind)

Formulate and solve the economic dispatch problem given the cost of generation 
for the two thermal generators and the cost of wind generation.
"""
function solve_economic_dispatch(;
        cost_of_thermal = COST_GENERATION, 
        cost_of_wind = COST_WIND)

    economic_dispatch = Model(GLPK.Optimizer)
    
    # Define decision variables    
    @variables(economic_dispatch, begin
        g[i=1:2]  # Thermal generation (MW).
        w >= 0  # Wind power (MW).
    end)

    # Define the objective function
    @objective(economic_dispatch, Min, 
        sum(cost_of_thermal[i] * g[i] for i in 1:2) + cost_of_wind * w
    )

    # Define the constraint on the maximum and minimum power output of each generator.
    for i in 1:2
        @constraint(economic_dispatch, g[i] <= GENERATION_MAX[i])
        @constraint(economic_dispatch, g[i] >= GENERATION_MIN[i])
    end
    
    @constraints(economic_dispatch, begin
        # Define the constraint on the wind power injection
        w <= WIND_MAX
        # Define the power balance constraint
        sum(g[i] for i in 1:2) + w == DEMAND
    end)

    # Solve statement
    optimize!(economic_dispatch)
    
    # Return the optimal value of the objective function and its minimizers
    # as a NamedTuple.
    return (
        generation = value.(g), 
        wind_generation = value(w),
        wind_spillage = WIND_MAX - value(w),
        cost = objective_value(economic_dispatch)
    )
end

# Solve the economic dispatch problem
solution = solve_economic_dispatch()

println("Dispatch")
println("   Generators: ", solution.generation, " MW")
println("         Wind: ", solution.wind_generation, " MW")
println("Wind spillage: ", solution.wind_spillage, " MW") 
println("----------------------------------")
println("Total cost: \$", solution.cost)  

### Economic dispatch with adjustable incremental costs

In the following exercise we introduce a manipulator to vary the cost of wind generation and observe its impact the total cost, dispatch of generators G1 and G2, utilization of available wind under different values of the incremental cost of generator G1.

In [ ]:
@manipulate for cost_of_wind in COST_WIND .* (1:0.1:3.5)
    solutions = Any[]
    cost_of_g1 = COST_GENERATION[1] .* (0.5:0.01:3.0)
    for c_g1 in cost_of_g1
        # update the incremental cost of the first generator at every iteration
        solution = solve_economic_dispatch(
            cost_of_thermal = [c_g1, COST_GENERATION[2]],
            cost_of_wind = cost_of_wind
        )
        push!(solutions, solution)
    end
    
    # Plot the outputs
    plot(
        # Plot the total cost
        plot(cost_of_g1, [sol.cost for sol in solutions],
            ylabel = "Total cost",
            ylims = (50000, 200000)
        ),
        # Plot the power output of Generator 1
        plot(cost_of_g1, [sol.generation[1] for sol in solutions],
            ylabel = "Dispatch: G1",
            ylims = (0, 1100)
        ),
        # Plot the power output of Generator 2    
        plot(cost_of_g1, [sol.generation[2] for sol in solutions],
            ylabel = "Dispatch: G2",
            ylims = (0, 1600)
        ),
        # Plot the wind power output
        plot(cost_of_g1, [sol.wind_generation for sol in solutions],
            ylabel = "Dispatch: Wind",
            ylims = (0, 250)
        ),
        legend = false,
        xlabel = "Cost of G1"
    )
end

## Nonlinear example

JuMP can also be used to solve non-linear problems (NLP). We saw a brief hint of this earlier when we used Ipopt and the `@NLobjective` macro. In the next example, we add nonlinearity to the cost of the second generator. The new cost of generation is
$$\text{generation_cost} = c_1^g\times g_1 + \frac{c_2^g\times  g_2^{1.5}}{\sqrt{1000}}.$$


In [ ]:
"""
    solve_economic_dispatch(; cost_of_thermal::Vector, cost_of_wind)

Formulate and solve the economic dispatch problem given the cost of generation 
for the two thermal generators and the cost of wind generation.
"""
function solve_nonlinear_economic_dispatch(;
        cost_of_thermal = COST_GENERATION, 
        cost_of_wind = COST_WIND)
    economic_dispatch = Model(Ipopt.Optimizer)
    set_optimizer_attribute(economic_dispatch, "print_level", 0)
    
    @variables(economic_dispatch, begin
        g[i=1:2] >= 0
        w >= 0
    end)

    # ===========================================================
    # You can write out nonlinear expression in the @NLobjective macro
    # The same also applies for @NLconstraint.
    @NLobjective(economic_dispatch, Min,
        cost_of_thermal[1] * g[1] + 
        cost_of_thermal[2] * g[2]^1.5 / sqrt(1000) + 
        cost_of_wind * w)
    # ===========================================================
    # Look! This bit changed.
    function generator_cost(g1, g2)
        return cost_of_thermal[1] * g1 + cost_of_thermal[2] * g2^1.5 / sqrt(1000)
    end
    JuMP.register(economic_dispatch, :generator_cost, 2, generator_cost, autodiff=true)
    @NLobjective(economic_dispatch, Min, 
        generator_cost(g[1], g[2]) + cost_of_wind * w)
    # ===========================================================
    
    for i in 1:2
        @constraint(economic_dispatch, g[i] <= GENERATION_MAX[i])
        @constraint(economic_dispatch, g[i] >= GENERATION_MIN[i])
    end    
    @constraints(economic_dispatch, begin
        w <= WIND_MAX
        sum(g[i] for i in 1:2) + w == DEMAND
    end)
    optimize!(economic_dispatch)
    return (
        generation = value.(g), 
        wind_generation = value(w),
        wind_spillage = WIND_MAX - value(w),
        cost = objective_value(economic_dispatch)
    )
end

In [ ]:
@manipulate for cost_of_wind in COST_WIND .* (1:0.2:3.5)
    solutions = Any[]
    cost_of_g1 = COST_GENERATION[1] .* (0.5:0.1:3.0)
    for c_g1 in cost_of_g1
        # update the incremental cost of the first generator at every iteration
        solution = solve_nonlinear_economic_dispatch(
            cost_of_thermal = [c_g1, COST_GENERATION[2]],
            cost_of_wind = cost_of_wind
        )
        push!(solutions, solution)
    end
    
    # Plot the outputs
    plot(
        # Plot the total cost
        plot(cost_of_g1, [sol.cost for sol in solutions],
            ylabel = "Total cost",
            ylims = (50000, 200000)
        ),
        # Plot the power output of Generator 1
        plot(cost_of_g1, [sol.generation[1] for sol in solutions],
            ylabel = "Dispatch: G1",
            ylims = (0, 1100)
        ),
        # Plot the power output of Generator 2    
        plot(cost_of_g1, [sol.generation[2] for sol in solutions],
            ylabel = "Dispatch: G2",
            ylims = (0, 1600)
        ),
        # Plot the wind power output
        plot(cost_of_g1, [sol.wind_generation for sol in solutions],
            ylabel = "Dispatch: Wind",
            ylims = (0, 250)
        ),
        legend = false,
        xlabel = "Cost of G1"
    )
end